In [17]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
%ls d3-3.1.0/

LICENSE*             d3.min.js*           package.json*
Makefile*            globals.js*          src/
README.md*           index-browserify.js* test/
component.json*      index.js*
d3.js*               lib/


In [19]:
%%javascript

var hello = 'hello'
console.log(hello)

<IPython.core.display.Javascript object>

In [20]:
from IPython.core.display import display, Javascript

In [21]:
require_js = '''
    if (window.d3 === undefined){{   
        require.config({{ paths: {{d3: "http://d3js.org/d3.v3.min"}} }});
        require(['d3'], function(d3) {{
            window.d3 = d3;
            console.log(window.d3);
        }});
    }};
'''
display(Javascript(require_js.format()))

<IPython.core.display.Javascript object>

In [22]:
%%html

<div id="d3-example"></div>

<style>
.node {stroke: #fff; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

In [23]:
%%html

<div class='bar'></div>

<style>
div.bar {
    display: inline-block;
    width: 20px;
    height: 75px;   /* We'll override height later */
    background-color: teal;
}
</style>

In [24]:
%%javascript
var dataset = [ 5, 10, 15, 20, 25 ];

d3.select("#d3-example").selectAll("div").style("height", function(d) {
    var barHeight = d * 5;  //Scale up by factor of 5
    return barHeight + "px";
}).style("margin-right", "2px");

<IPython.core.display.Javascript object>

In [26]:
%%javascript
var w = 500;
var h = 50;
var dataset = [ 5, 10, 15, 20, 25 ];

var svg = d3.select("#d3-example")
            .append("svg")
            .attr("width", w)   
            .attr("height", h);
            
var circles = svg.selectAll("circle")
                 .data(dataset)
                 .enter()
                 .append("circle");

circles.attr("cx", function(d, i) {
            return (i * 50) + 25;
        })
       .attr("cy", h/2)
       .attr("r", function(d) {
            return d;
       })
       .attr("fill", function(d) {
            return "rgb(0, 0, " + (d * 10) + ")";
       });

<IPython.core.display.Javascript object>

In [11]:
%%javascript
var w = 500;
var h = 250;
var barPadding = 1;
var dataset = [ 5, 10, 13, 19, 21, 25, 22, 18, 15, 13,
                11, 12, 15, 20, 18, 17, 16, 18, 23, 25 ];

var svg = d3.select("#d3-example")
            .append("svg")
            .attr("width", w)   
            .attr("height", h);

var bars = svg.selectAll("div")
    .data(dataset)
    .enter()
    .append("div")
    .attr("class", "bar")
    .style("height", function(d) {
        var barHeight = d * 5;
        return barHeight + "px";
    });

bars.attr("fill", function(d) {
            return "rgb(0, 0, " + (d * 10) + ")";
       })
       .text(function(d) {
            return d;
       })
       .attr("x", function(d, i) {
            return i * (w / dataset.length) + (w / dataset.length - barPadding) / 2;
       })
       .attr("y", function(d) {
            return h - (d * 4) + 14;  
       })
       .attr("font-family", "sans-serif")
       .attr("font-size", "11px")
       .attr("fill", "white")
       .attr("text-anchor", "middle");

<IPython.core.display.Javascript object>

We want to visualize tSNE using D3, that's a great stuff

In [32]:
# %load "/Volumes/STORAGE/data/tools/tsne.py"
#
#  tsne.py
#  
# Implementation of t-SNE in Python. The implementation was tested on Python 2.5.1, and it requires a working 
# installation of NumPy. The implementation comes with an example on the MNIST dataset. In order to plot the
# results of this example, a working installation of matplotlib is required.
# The example can be run by executing: ipython tsne.py -pylab
#
#
#  Created by Laurens van der Maaten on 20-12-08.
#  Copyright (c) 2008 Tilburg University. All rights reserved.

import numpy as Math
import pylab as Plot
	
def Hbeta(D = Math.array([]), beta = 1.0):
	"""Compute the perplexity and the P-row for a specific value of the precision of a Gaussian distribution."""
	
	# Compute P-row and corresponding perplexity
	P = Math.exp(-D.copy() * beta);
	sumP = sum(P);
	H = Math.log(sumP) + beta * Math.sum(D * P) / sumP;
	P = P / sumP;
	return H, P;
	
	
def x2p(X = Math.array([]), tol = 1e-5, perplexity = 30.0):
	"""Performs a binary search to get P-values in such a way that each conditional Gaussian has the same perplexity."""

	# Initialize some variables
	print "Computing pairwise distances..."
	(n, d) = X.shape;
	sum_X = Math.sum(Math.square(X), 1);
	D = Math.add(Math.add(-2 * Math.dot(X, X.T), sum_X).T, sum_X);
	P = Math.zeros((n, n));
	beta = Math.ones((n, 1));
	logU = Math.log(perplexity);
    
	# Loop over all datapoints
	for i in range(n):
	
		# Print progress
		if i % 500 == 0:
			print "Computing P-values for point ", i, " of ", n, "..."
	
		# Compute the Gaussian kernel and entropy for the current precision
		betamin = -Math.inf; 
		betamax =  Math.inf;
		Di = D[i, Math.concatenate((Math.r_[0:i], Math.r_[i+1:n]))];
		(H, thisP) = Hbeta(Di, beta[i]);
			
		# Evaluate whether the perplexity is within tolerance
		Hdiff = H - logU;
		tries = 0;
		while Math.abs(Hdiff) > tol and tries < 50:
				
			# If not, increase or decrease precision
			if Hdiff > 0:
				betamin = beta[i];
				if betamax == Math.inf or betamax == -Math.inf:
					beta[i] = beta[i] * 2;
				else:
					beta[i] = (beta[i] + betamax) / 2;
			else:
				betamax = beta[i];
				if betamin == Math.inf or betamin == -Math.inf:
					beta[i] = beta[i] / 2;
				else:
					beta[i] = (beta[i] + betamin) / 2;
			
			# Recompute the values
			(H, thisP) = Hbeta(Di, beta[i]);
			Hdiff = H - logU;
			tries = tries + 1;
			
		# Set the final row of P
		P[i, Math.concatenate((Math.r_[0:i], Math.r_[i+1:n]))] = thisP;
	
	# Return final P-matrix
	print "Mean value of sigma: ", Math.mean(Math.sqrt(1 / beta))
	return P;
	
	
def pca(X = Math.array([]), no_dims = 50):
	"""Runs PCA on the NxD array X in order to reduce its dimensionality to no_dims dimensions."""

	print "Preprocessing the data using PCA..."
	(n, d) = X.shape;
	X = X - Math.tile(Math.mean(X, 0), (n, 1));
	(l, M) = Math.linalg.eig(Math.dot(X.T, X));
	Y = Math.dot(X, M[:,0:no_dims]);
	return Y;


def tsne(X = Math.array([]), no_dims = 2, initial_dims = 50, perplexity = 30.0):
	"""Runs t-SNE on the dataset in the NxD array X to reduce its dimensionality to no_dims dimensions.
	The syntaxis of the function is Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array."""
	
	# Check inputs
	if X.dtype != "float64":
		print "Error: array X should have type float64.";
		return -1;
	#if no_dims.__class__ != "<type 'int'>":			# doesn't work yet!
	#	print "Error: number of dimensions should be an integer.";
	#	return -1;
	
	# Initialize variables
	X = pca(X, initial_dims);
	(n, d) = X.shape;
	max_iter = 1000;
	initial_momentum = 0.5;
	final_momentum = 0.8;
	eta = 500;
	min_gain = 0.01;
	Y = Math.random.randn(n, no_dims);
	dY = Math.zeros((n, no_dims));
	iY = Math.zeros((n, no_dims));
	gains = Math.ones((n, no_dims));
	
	# Compute P-values
	P = x2p(X, 1e-5, perplexity);
	P = P + Math.transpose(P);
	P = P / Math.sum(P);
	P = P * 4;									# early exaggeration
	P = Math.maximum(P, 1e-12);
	
	# Run iterations
	for iter in range(max_iter):
		
		# Compute pairwise affinities
		sum_Y = Math.sum(Math.square(Y), 1);		
		num = 1 / (1 + Math.add(Math.add(-2 * Math.dot(Y, Y.T), sum_Y).T, sum_Y));
		num[range(n), range(n)] = 0;
		Q = num / Math.sum(num);
		Q = Math.maximum(Q, 1e-12);
		
		# Compute gradient
		PQ = P - Q;
		for i in range(n):
			dY[i,:] = Math.sum(Math.tile(PQ[:,i] * num[:,i], (no_dims, 1)).T * (Y[i,:] - Y), 0);
			
		# Perform the update
		if iter < 20:
			momentum = initial_momentum
		else:
			momentum = final_momentum
		gains = (gains + 0.2) * ((dY > 0) != (iY > 0)) + (gains * 0.8) * ((dY > 0) == (iY > 0));
		gains[gains < min_gain] = min_gain;
		iY = momentum * iY - eta * (gains * dY);
		Y = Y + iY;
		Y = Y - Math.tile(Math.mean(Y, 0), (n, 1));
		
		# Compute current value of cost function
		if (iter + 1) % 10 == 0:
			C = Math.sum(P * Math.log(P / Q));
			print "Iteration ", (iter + 1), ": error is ", C
			
		# Stop lying about P-values
		if iter == 100:
			P = P / 4;
			
	# Return solution
	return Y;
		
	
# if __name__ == "__main__":
	# print "Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset."
	# print "Running example on 2,500 MNIST digits..."
	# X = Math.loadtxt("mnist2500_X.txt");
	# labels = Math.loadtxt("mnist2500_labels.txt");
	# Y = tsne(X, 2, 50, 20.0);
	# Plot.scatter(Y[:,0], Y[:,1], 20, labels);


In [34]:
# load the MNIST data set 
X = np.loadtxt("/Volumes/STORAGE/data/tools/tsne_python/mnist2500_X.txt")
labels = np.loadtxt("/Volumes/STORAGE/data/tools/tsne_python/mnist2500_labels.txt")
Y = tsne(X, no_dims=2, initial_dims=87, perplexity=20.0)  # 87 dimensions in PCA to keep 90% of the variance

Preprocessing the data using PCA...
Computing pairwise distances...
Computing P-values for point  0  of  2500 ...
Computing P-values for point  500  of  2500 ...
Computing P-values for point  1000  of  2500 ...
Computing P-values for point  1500  of  2500 ...
Computing P-values for point  2000  of  2500 ...
Mean value of sigma:  2.98303641893
Iteration  10 : error is  19.3708714655
Iteration  20 : error is  18.3409514299
Iteration  30 : error is  16.5550734566
Iteration  40 : error is  16.0133728771
Iteration  50 : error is  16.0328585093
Iteration  60 : error is  16.0270136535
Iteration  70 : error is  15.9733132032
Iteration  80 : error is  15.9455976394
Iteration  90 : error is  15.9556588953
Iteration  100 : error is  15.9910730603
Iteration  110 : error is  2.11869358059
Iteration  120 : error is  1.68281307748
Iteration  130 : error is  1.47694558909
Iteration  140 : error is  1.35425460656
Iteration  150 : error is  1.27161190307
Iteration  160 : error is  1.21325013234
Iteratio

/Users/tubi/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:78: ComplexWarning: Casting complex values to real discards the imaginary part


In [40]:
Y.shape   # just basically a scatter plot

(2500, 2)

In [67]:
labels.shape

(2500,)

In [262]:
# export the data into csv
data = pd.DataFrame(Y, columns=['X', 'Y'])
data['raw'] = ''
for i, row in enumerate(X):
    data.raw[i] = ''.join(pd.Series(row).astype('int').astype('str'))

data.X = data.X - data.X.min() + 20 # margin 20
data.Y = data.Y - data.Y.min() + 20
data['label'] = labels
data.label = data.label.astype('int')
data.to_csv("tsne_matrix.csv", index=False)

/Users/tubi/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
%ls

D3_Basics.ipynb  d3-3.1.0/        tsne_matrix.csv


In [13]:
%system head -n 2 tsne_matrix.csv

['X,Y,raw,label',
 '70.78195276260732,75.58002913890462,1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110011111111111111111111111111001111111111111111111111111000111111111100111111111111100011111111100001111111111100001111111110000011111111110000111111111000001111111110000011111111100000001111111000001111111100000000011111000001111111110000000000111100000111111111000001000001100000111111111100001111000010000011111111110000011100001000011111111111000001110000000001111111111100011111100000001111111111110001111111000001111111111111000111111111001111111111111100011111111111111111111111110001111111111111111111111111001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111,5']

In [14]:
%%html
<div id='tsne'></div>

<div id='digit' height='60' width='60' style='position:relative'>
    <canvas height='58' width='58'></canvas>
</div>

<style>


.tooltip {   
    position:       absolute;
    
    box-shadow:     0 0 5px #999999;
    text-align:     center; 
    width:          60px;
    padding:        1px;             
    font:           12px sans-serif;        
    border:         1px;      
    border-radius:  8px;           
    pointer-events: none;         
}

</style>

In [15]:
%%javascript

var tsne_data = [];
var w = 500;
var h = 500;

// Reading the data from csv first 
d3.csv("tsne_matrix.csv", function (data) {
    tsne_data = data.map(function (row) { return { X: +row['X'], Y: +row['Y'], label: +row['label'], raw: row['raw'] }; });
    console.log(tsne_data[0]);   // must be inside the callback
    
    var color = d3.scale.ordinal().domain(d3.range(10))
        .range(['#CFD4C5', '#EFB9CB', '#C287E8', '#FCB97D', '#EDD892', '#C6B89E', '#AABA9E', '#50808E', '#F4FDD9', '#FFE8C2'])

    // Creating a frame 
    var svg = d3.select('#tsne').append('svg').attr('width', w).attr('height', h);

    var show_digit = function (canvas, string) {
        var ctx = canvas.getContext('2d'); 
        ctx.fillStyle = "#000";                 // Fill with white first
        ctx.fillRect(0, 0, d_len*step, d_len*step);
        ctx.fillStyle = "#FFF";
        for (var i = 0; i < d_len*step; i += step) {  // Loop through each character
            for (var j = 0; j < d_len*step; j += step) {
                if (string[(i*d_len+j)/step] == '1')     // If the character is one,
                    ctx.fillRect(i, j, step, step);  // fill the pixel with black
            }
        }
    };
    
    var d_len = 28;  // dimension of digit
    var step  = 2;   // scale the image 
    
    var tooltip = d3.select('#digit').attr('class', 'tooltip').style('opacity', 0); 
    
    svg.selectAll('circle')
        .data(tsne_data)
        .enter().append('circle')   // after added circle
        .attr("r", 3)
        .attr("cx", function(d) { return d.X*3 ; })
        .attr("cy", function(d) { return d.Y*3 ; })
        .style("fill", function(d) { return color(d.label); })
        .on('mouseover', function (d) {
            
            var canvas = tooltip.select('canvas')[0][0];  // one level for array
            show_digit(canvas, d.raw);
        
            tooltip.transition().duration(100)
                .style("opacity", .9)
                .style("left", (d3.event.pageX - 300) + "px")
                .style("top", (d3.event.pageY - 500) + "px");                
        })
        .on('mouseout', function () {
            tooltip.transition().duration(100).style("opacity", 0); 
        });
    
}); 


<IPython.core.display.Javascript object>

In [16]:
%%html
<canvas id='can' width="56" height="56"></canvas>

In [ ]:
# todo: fix the position of the mouse when we point onto the top
# effect of the circle when hover
# ability to zoom in
# do the analysis 